In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from PIL import Image

from pathlib import Path

df = pd.DataFrame(columns=['frame', 'brightness', 'valence', 'max', 'x', 'y', 'area' ])

def analyze_frame(frame_num):
    file_name = "frames/" + str(frame_num) + ".png"
    path = Path(file_name)
    if not path.is_file():
        print("no file, get last")
        file_name = "frames/" + str(frame_num-1) + ".png"

    image = Image.open(os.path.join(sys.path[0], file_name))
    greyscale_image = image.convert('L')
    histogram = greyscale_image.histogram()
    pixels = sum(histogram)
    brightness = scale = len(histogram)
    for index in range(0, scale):
        ratio = histogram[index] / pixels
        brightness += ratio * (-scale + index)
    hist = np.array(histogram)

    valence =np.count_nonzero(hist)
    max_b = hist.argmax(axis=0)

    
    image = cv.imread('frames/'+str(frame_num)+'.png')
    


    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (5, 5),cv.BORDER_DEFAULT)
    ret, thresh = cv.threshold(blur, 100, 255, cv.THRESH_BINARY_INV)
    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    cx=0
    cy=0
    area = 0
    if len(contours)>1 :
        M = cv.moments(contours[-2])
        if M['m00'] == cv.contourArea(contours[-2]):
            
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            
            cv.drawContours(image, [contours[-2]], -1, (0, 255, 0), 2)
            cv.putText(image, str(frame_num), (cx-150, cy-75),
                       cv.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
            cv.putText(image, "a: "+str(M['m00']), (cx-150, cy+135),
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv.putText(image, "b: " + str(round(brightness / scale,2)), (cx-150, cy+175),
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv.imwrite("frames/contours/"+str(frame_num)+".png", image)
            area = M['m00']

    
            oimg = Image.open(os.path.join(sys.path[0], "frames/contours/"+str(frame_num)+".png"))
            left = cx-200
            top = cy-200
            right = cx+200
            bottom = cy+200
            im1 = oimg.crop((left, top, right, bottom))
            im1.save("./frames/gif/"+ "{:04d}".format(frame_num) +".png")
#             df.loc[len(df)] = [frame_num, M['m00'], cx, cy]
    else:
        print("did not find area for " + str(frame_num))
        
    df.loc[len(df)] = [frame_num, (brightness / scale), valence, max_b, cx, cy, area]


frames = []
for j in range(0,100):
    print(j)
    i = j+1
    analyze_frame(i)

    
df.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


,frame,brightness,valence,max,x,y,area
0,1.0,0.010983,90.0,3.0,385.0,290.0,59.0
1,2.0,0.010984,93.0,3.0,386.0,293.0,62.0
2,3.0,0.010983,100.0,3.0,387.0,294.0,62.0
3,4.0,0.010976,92.0,3.0,388.0,294.0,60.0
4,5.0,0.010969,100.0,3.0,389.0,294.0,61.0
